# WebScrapping Mollet del Valles - Covid 19

## Introducción

La intención de esta notebook, es la de explicar un script python que cree al principio de la pandemia para recibir de forma automática (via crontab en un servidor linux que tengo en una jetson nano) las notícias locales (de mi pueblo) sobre covid recogidas en diferentes medios locales.

## Medios Locales 'Scrapeados'

- Listado de medios incluidos en el script

[Nació Granollers](https://www.naciodigital.cat/naciogranollers/)

[Som Mollet](https://www.sommollet.cat/)

[El 9nou Vallès Oriental](https://el9nou.cat/valles-oriental/)

[Mollet a Mà](https://www.clicama.cat/)

[Ajuntament Mollet del Vallès](https://www.molletvalles.cat/)

[Revista del Vallès](https://revistadelvalles.es/)

## Pasos
El script principalmente hace:
- Scrapea el medio local con la dirección que me pareció mas fácil de scrapear.
- Del contenido de las noticias busca 2 palabras clave, 'covid' 'mollet'.
- Si se cumplen ambas añade el título de la notícia, parte del texto (se puede configurar) y el link de la notícia en un dataframe, que finalmente formateará como html y enviará por correo.

## Script

Librerías

In [3]:
import not_lib as nt #librería copiada (no encuentro la ref), paso al final
import pandas as pd
import clean_lib as cl #idem not_lib, sirven para dejar el texto legible
from mail_lib import enviar_mail #librería propia, paso detalle al final
import urllib.request
from bs4 import BeautifulSoup

Creo el dataframe de 'trabajo' en el que exportaré las notícias al final y que enviaré por correo electrónico en formato 'html', no es necesario definirlo, pero a mi me ayuda a visualizar que datos utilizo y para que etc...

In [5]:
noticias = pd.DataFrame(columns=['Titulo','Link','Texto','Origen'])
pd.set_option('display.max_colwidth', None) #Con esto no truncamos el link

### Ahora iremos medio a medio informando el DataFrame

**Nota** La url seleccionada, se ha realizado via ensayo y error (sin mucho sufrimiento), simplemente intentando buscar un punto de la web del medio que fuera facil conseguir las notícias y el link al contenido (busca primero todas las noticias de cabecera y después mira el contenido de cada una al completo).

#### El 9nou

In [6]:
url = 'https://el9nou.cat/valles-oriental/'
soup = nt.f_get_data(url)
art = soup.find(class_='noticies-portada')

items = art.find_all('article')

for i,item in enumerate(items):
    url_item = item.find('a')['href']
    tit_item = item.find('h1').getText()
    soup2 = nt.f_get_data(url_item)
    text_art = soup2.find(class_="col-md-12 marge")
    body = text_art.find_all('p')
    texto = ''
    origen = '9nou'
    for body_item in body:
        texto = texto + body_item.getText()
    noticias.loc[len(noticias)]=[tit_item,url_item,texto,origen]

#### Nació Granollers

In [8]:
url2 = 'https://www.naciodigital.cat/naciogranollers/rss'
request = urllib.request.Request(url2, headers={'User-Agent': 'Mozilla/5.0'})
response = urllib.request.urlopen(request, timeout=20)
content = response.read()
# Soup object (identify tags, etc..)
soup = BeautifulSoup(content, "lxml")
art = soup.find_all('item')
for art_item in art:
    title = art_item.find('title').getText()
    link  = art_item.find('guid').getText()
    soup2 = nt.f_get_data(link)
    body = soup2.find_all(class_='amp_textnoticia')
    texto = ''
    origen = 'nacio'
    for body_item in body:
       texto = texto + body_item.getText()
    noticias.loc[len(noticias)]=[title,link,texto,origen]

#### Som Mollet

In [9]:
url3 = 'https://www.sommollet.cat/rss'
request = urllib.request.Request(url3, headers={'User-Agent': 'Mozilla/5.0'})
response = urllib.request.urlopen(request, timeout=20)
content = response.read()
# Soup object (identify tags, etc..)
soup = BeautifulSoup(content, "lxml-xml")
art = soup.find_all('item')
for art_item in art:
    title = art_item.find('title').getText()
    link  = art_item.find('guid').getText()
    soup2 = nt.f_get_data(link)
    body = soup2.find_all(class_='interior-main__content')
    origen = 'sommollet'
    texto = ''
    for body_item in body:
       texto = texto + body_item.getText()
    noticias.loc[len(noticias)]=[title,link,texto,origen]

#### Mollet a Mà

In [10]:
url4 = 'https://www.clicama.cat/rss'
request = urllib.request.Request(url4, headers={'User-Agent': 'Mozilla/5.0'})
response = urllib.request.urlopen(request, timeout=20)
content = response.read()
# Soup object (identify tags, etc..)
soup = BeautifulSoup(content, "lxml-xml")
art = soup.find_all('item')
for art_item in art:
    title = art_item.find('title').getText()
    link  = art_item.find('guid').getText()
    soup2 = nt.f_get_data(link)
    body = soup2.find_all(class_='content-data')
    origen = 'molletama'
    texto = ''
    for body_item in body:
       texto = texto + body_item.getText()
    noticias.loc[len(noticias)]=[title,link,texto,origen]

#### Ajuntament de Mollet del Vallès

In [11]:
url5 = 'https://www.molletvalles.cat/continguts-es-es/actualitat-es-es/notcies-es-es/'
request = urllib.request.Request(url5, headers={'User-Agent': 'Mozilla/5.0'})
response = urllib.request.urlopen(request, timeout=20)
content = response.read()
# Soup object (identify tags, etc..)
soup = BeautifulSoup(content, "html.parser")
art = soup.find_all(class_='capsa_noticies_item')
for i,art_item in enumerate(art):
    link = art_item.find('a')['href']
    origen = 'ajuntament'
    if art_item.find('h3') != None:
        title = art_item.find('h3').getText()
        soup2 = nt.f_get_data(link)
        body = soup2.find_all(style='text-align:justify')
        texto = body[0].getText()
        noticias.loc[len(noticias)]=[title,link,texto,origen]

#### Revista del Vallès

In [12]:
url6 = 'https://revistadelvalles.es/feed/'
request = urllib.request.Request(url6, headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.75.14 (KHTML, like Gecko) Version/7.0.3 Safari/7046A194A'})
response = urllib.request.urlopen(request, timeout=20)
content = response.read()
# Soup object (identify tags, etc..)
soup = BeautifulSoup(content, "lxml-xml")
art = soup.find_all('item')
for art_item in art:
    title = art_item.find('title').getText()
    link  = art_item.find('guid').getText()
    soup2 = nt.f_get_data(link)
    body = soup2.find_all(class_='td-post-content tagdiv-type')
    origen = 'revistavalles'
    texto = ''
    for body_item in body:
       texto = texto + body_item.getText()
    noticias.loc[len(noticias)]=[title,link,texto,origen]

### Definimos la función de búsqueda (está en la librería not_lib, aquí pongo el detalle)

In [13]:
import urllib.request
from bs4 import BeautifulSoup

def f_get_data(url):
    #Import html information
    request = urllib.request.Request(url,headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.75.14 (KHTML, like Gecko) Version/7.0.3 Safari/7046A194A'})
    response = urllib.request.urlopen(request, timeout=20)
    content = response.read()
    #Soup object (identify tgs, etc..)
    soup = BeautifulSoup(content, "html.parser")
    return soup

def f_mollet_corona(text):
    value = text.find('mollet')
    value2 = text.find('coronavirus')
    if value > 0 and value2 > 0:
        return 1
    else:
        return 0

def busca_mollet(noticias):
    encontrar = lambda x: f_mollet_corona(x)
    noticias['Importante']=noticias.Clean.apply(encontrar)
    noimportante = noticias[noticias['Importante']==0].index
    noticias.drop(labels=noimportante,inplace=True)
    return noticias

### Definimos la función de limpieza de texto etc... para facilitar la búsqueda (poner en minúsculas, quitar acentos, símbolos, etc..), está en la librería clean_lib

In [14]:
# Apply a first round of text cleaning techniques
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?¿\]\%', ' ', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

# Apply a second round of cleaning
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…«»]', '', text)
    text = re.sub('\n', ' ', text)
    return text

### Aplicamos las librerías de búsqueda y filtramos el conjunto de notícias que tenemos con las que nos interesan

In [15]:
#Limpiamos
round1 = lambda x: cl.clean_text_round1(x)
noticias['Clean'] = noticias.Texto.apply(round1)
round2 = lambda x: cl.clean_text_round2(x)
noticias['Clean'] = noticias.Clean.apply(round2)

resultado = nt.busca_mollet(noticias)
resultado.drop(labels=['Texto','Clean','Importante','Origen'],axis=1,inplace=True)
resultado.reset_index(drop=True, inplace=True)

### Finalmente Enviamos el df via mail, paso en primer termino las funciones que hay en mail_lib y después el código principal

In [16]:
#Libreria para envio de correos html.
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

def enviar_mail(direccion,df):
    # Correo de acceso al servidor
    MY_ADDRESS = 'mailuser'
    # Password de acceso a la cuenta de email
    PASSWORD = 'password'

    # Configurar el servidor de correo
    s = smtplib.SMTP(host='smtp.host.com', port=587) # servidor y puerto
    s.starttls() # Conexion tls
    s.login(MY_ADDRESS, PASSWORD) # Iniciar sesion con los datos de acceso al servidor SMTP

    # Crear el Mensaje
    msg = MIMEMultipart('alternative')
    text = 'Hi All,\n'
    html = '''<h3>Últimas notícias sobre #coronavirus y #Mollet.</h3><p><p>
    Las notícias pueden estar repetidas, se busca todas las que los medios tienen publicadas en sus paginas principales<p>{}<p><p>Sergi García<p>Python Analysis Tools'''.format(df.to_html())
    # Record the MIME types of both parts - text/plain and text/html.
    part1 = MIMEText(text, 'plain')
    part2 = MIMEText(html, 'html')
    # Attach parts into message container.
    msg.attach(part1)
    msg.attach(part2)

    # Configurar los parametros del mensaje
    msg['From']=MY_ADDRESS
    msg['To']= direccion
    msg['Subject']="Notícias Locales - #Coronaviurs #Mollet"

    # Enviar el mensaje
    s.send_message(msg)
    del msg

    # Finaliar sesion SMTP
    s.quit()

In [ ]:
#Enviar email
to = 'mail@gmail.com'
enviar_mail(to,resultado)